In [1]:
import pandas as pd
import numpy as np
import random
import bisect

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

import matplotlib.pyplot as plt
from tqdm import tqdm

import xgboost as xgb

In [2]:
RANDOM_SEED = 99

In [3]:
train_df = pd.read_parquet("../data/HD_data/train_v2_remove.parquet")
test_df = pd.read_parquet("../data/HD_data/test_v2_remove.parquet")

In [4]:
train_x = train_df.drop(columns=["CI_HOUR"])
train_y = train_df["CI_HOUR"]

In [5]:
def create_model(param):
    return xgb.XGBRegressor(
        random_state=RANDOM_SEED,
        tree_method="gpu_hist",
        eval_metric="mae",
        early_stopping_rounds=50,
        **param
    )

In [6]:
def train_model(param, X, y, X_test, trial=None):
    skf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

    val_scores = []
    y_tests = []
    models = []

    for idx, (train_idx, val_idx) in enumerate(skf.split(X, y)):

        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

        model = create_model(param)
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=500)

        y_hat_val = model.predict(X_val)
        score = mean_absolute_error(y_val.values, y_hat_val)
        val_scores.append(score)
        print(f'Fold: {idx+1}/5 score = {score:.5f}')

        y_tests.append(model.predict(X_test))
        models.append(model)

        if trial:
            trial.report(score, idx)

            if trial.should_prune():
                raise optuna.TrialPruned()

    return val_scores, y_tests, models

In [7]:
def objective_xgb(trial):
  param = {
    "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
    "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
    "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    "subsample": trial.suggest_float("subsample", 0.1, 1.0),
    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
    "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
    "max_depth": trial.suggest_int("max_depth", 3, 9),
    "min_child_weight": trial.suggest_int("min_child_weight", 2, 10),
    "eta": trial.suggest_float("eta", 1e-8, 1.0, log=True),
    "gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True),
    "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
  }

  if param["booster"] == "dart":
    param['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
    param['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
    param['rate_drop'] = trial.suggest_float('rate_drop', 1e-8, 1.0, log=True)
    param['skip_drop'] = trial.suggest_float('skip_drop', 1e-8, 1.0, log=True)

  val_scores, y_tests, models = train_model(param, train_x, train_y, test_df, trial)

  return sum(val_scores) / len(val_scores)

In [8]:
study = optuna.create_study(
    sampler=TPESampler(seed=RANDOM_SEED),
    direction='minimize',
    study_name='xgb_tuning',
    pruner=HyperbandPruner(
      min_resource=1, max_resource=8, reduction_factor=3
    ),
)

study.optimize(objective_xgb, n_trials=100)

[I 2023-10-04 21:10:15,751] A new study created in memory with name: xgb_tuning


[0]	validation_0-mae:61.84817	validation_1-mae:61.50736
[299]	validation_0-mae:61.84802	validation_1-mae:61.50721
Fold: 1/5 score = 61.50721
[0]	validation_0-mae:61.79480	validation_1-mae:61.72088
[299]	validation_0-mae:61.79464	validation_1-mae:61.72072
Fold: 2/5 score = 61.72072
[0]	validation_0-mae:61.72215	validation_1-mae:62.01147
[299]	validation_0-mae:61.72200	validation_1-mae:62.01131
Fold: 3/5 score = 62.01131
[0]	validation_0-mae:61.61728	validation_1-mae:62.43095
[299]	validation_0-mae:61.61712	validation_1-mae:62.43080
Fold: 4/5 score = 62.43080
[0]	validation_0-mae:61.91766	validation_1-mae:61.22940
[299]	validation_0-mae:61.91751	validation_1-mae:61.22925
Fold: 5/5 score = 61.22925


[I 2023-10-04 21:10:47,419] Trial 0 finished with value: 61.77985963888551 and parameters: {'booster': 'gbtree', 'lambda': 0.04017550942202233, 'alpha': 1.7847241527490656e-08, 'subsample': 0.8272449670283629, 'colsample_bytree': 0.6090556776494775, 'n_estimators': 300, 'max_depth': 3, 'min_child_weight': 10, 'eta': 1.1339812255777714e-08, 'gamma': 0.014399394343769443, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 61.77985963888551.


[0]	validation_0-mae:61.84814	validation_1-mae:61.50733
[899]	validation_0-mae:61.79957	validation_1-mae:61.45970
Fold: 1/5 score = 61.45970
[0]	validation_0-mae:61.79477	validation_1-mae:61.72085
[899]	validation_0-mae:61.74638	validation_1-mae:61.67248
Fold: 2/5 score = 61.67248
[0]	validation_0-mae:61.72212	validation_1-mae:62.01144
[899]	validation_0-mae:61.67374	validation_1-mae:61.96302
Fold: 3/5 score = 61.96302
[0]	validation_0-mae:61.61725	validation_1-mae:62.43092
[899]	validation_0-mae:61.56907	validation_1-mae:62.38215
Fold: 4/5 score = 62.38215
[0]	validation_0-mae:61.91763	validation_1-mae:61.22937
[899]	validation_0-mae:61.86909	validation_1-mae:61.18064
Fold: 5/5 score = 61.18064


[I 2023-10-04 22:10:27,673] Trial 1 finished with value: 61.7315965775972 and parameters: {'booster': 'dart', 'lambda': 1.4575798463318516e-05, 'alpha': 0.6189425995687715, 'subsample': 0.5719732438377161, 'colsample_bytree': 0.1842517839679773, 'n_estimators': 900, 'max_depth': 4, 'min_child_weight': 6, 'eta': 2.178476742104051e-06, 'gamma': 0.033817296080697445, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 1.4193450623251104e-07, 'skip_drop': 4.983900949448835e-07}. Best is trial 1 with value: 61.7315965775972.


...

[I 2023-10-05 02:10:21,255] Trial 97 finished with value: 50.31477197383074 and parameters: {'booster': 'gbtree', 'lambda': 5.56872230129885e-08, 'alpha': 0.000371403226741631, 'subsample': 0.2792091937562424, 'colsample_bytree': 0.9780100265440612, 'n_estimators': 600, 'max_depth': 7, 'min_child_weight': 9, 'eta': 0.06096762225647851, 'gamma': 1.4431488045123722e-08, 'grow_policy': 'lossguide'}. Best is trial 95 with value: 50.27740532536846.


[0]	validation_0-mae:58.86209	validation_1-mae:58.56604
[59]	validation_0-mae:55.04907	validation_1-mae:55.53702
Fold: 1/5 score = 50.16231
[0]	validation_0-mae:58.83102	validation_1-mae:58.76579
[60]	validation_0-mae:54.87227	validation_1-mae:55.39725
Fold: 2/5 score = 50.20060
[0]	validation_0-mae:58.82418	validation_1-mae:59.12009
[60]	validation_0-mae:54.94151	validation_1-mae:55.96201
Fold: 3/5 score = 50.65610
[0]	validation_0-mae:58.59386	validation_1-mae:59.39341
[60]	validation_0-mae:54.99351	validation_1-mae:55.97012
Fold: 4/5 score = 50.88058
[0]	validation_0-mae:58.92762	validation_1-mae:58.21454
[60]	validation_0-mae:55.19014	validation_1-mae:55.39768
Fold: 5/5 score = 49.82110


[I 2023-10-05 02:10:42,304] Trial 98 finished with value: 50.34413716040664 and parameters: {'booster': 'gbtree', 'lambda': 6.726688632906952e-08, 'alpha': 0.0003579024964076794, 'subsample': 0.2848618282213726, 'colsample_bytree': 0.9784245480478594, 'n_estimators': 600, 'max_depth': 7, 'min_child_weight': 9, 'eta': 0.05548547043336019, 'gamma': 1.9117986884337226e-08, 'grow_policy': 'lossguide'}. Best is trial 95 with value: 50.27740532536846.


[0]	validation_0-mae:58.29822	validation_1-mae:58.02020
[59]	validation_0-mae:55.20927	validation_1-mae:55.79503
Fold: 1/5 score = 50.27173
[0]	validation_0-mae:58.18995	validation_1-mae:58.12986
[58]	validation_0-mae:55.11283	validation_1-mae:55.66942
Fold: 2/5 score = 50.20815
[0]	validation_0-mae:58.20565	validation_1-mae:58.50707
[57]	validation_0-mae:55.23000	validation_1-mae:56.28330
Fold: 3/5 score = 50.68530
[0]	validation_0-mae:57.94235	validation_1-mae:58.73477
[59]	validation_0-mae:55.11271	validation_1-mae:56.22526
Fold: 4/5 score = 50.76180
[0]	validation_0-mae:58.29468	validation_1-mae:57.58253
[58]	validation_0-mae:55.36737	validation_1-mae:55.73751
Fold: 5/5 score = 49.84289


[I 2023-10-05 02:11:03,362] Trial 99 finished with value: 50.3539729936981 and parameters: {'booster': 'gbtree', 'lambda': 5.883746442312592e-08, 'alpha': 0.0003686863887981493, 'subsample': 0.27695466762638277, 'colsample_bytree': 0.9752128047111558, 'n_estimators': 600, 'max_depth': 7, 'min_child_weight': 9, 'eta': 0.06935202685260955, 'gamma': 1.3427515136088721e-08, 'grow_policy': 'lossguide'}. Best is trial 95 with value: 50.27740532536846.


In [ ]:
trial = study.best_trial

In [ ]:
print(trial.params)

In [ ]:
best_param = {
  'booster': 'dart',
  'lambda': 0.061283320910387504,
  'alpha': 0.12324096901577561,
  'subsample': 0.9445691245384257,
  'colsample_bytree': 0.7342252303767117,
  'n_estimators': 1000,
  'max_depth': 9,
  'min_child_weight': 5,
  'eta': 0.23144138974063544,
  'gamma': 0.24487575503353165,
  'grow_policy': 'depthwise',
  'sample_type': 'uniform',
  'normalize_type': 'tree',
  'rate_drop': 5.278642199662013e-06,
  'skip_drop': 0.10619978534337826
}

val_scores, y_tests, models = train_model(best_param, train_x, train_y, test_df)

In [ ]:
importance = models[-1].get_booster().get_score(importance_type="weight")
tuples = [(k, importance[k]) for k in importance]
tuples.sort(key=lambda x: x[1])

label, values = zip(*tuples)

plt.rcParams["font.family"] = "MalGun Gothic"

plt.barh(label, values)
plt.show()

In [ ]:
y_tests

In [ ]:
submit = pd.read_csv("./data/sample_submission.csv")
submit["CI_HOUR"] = [i if i > 0 else 0 for i in np.mean(y_tests, axis=0)]
submit.to_csv("./csv/xgboost_optuna.csv", index=False)